In [37]:
import pandas as pd
import datetime as dt
import teradata
import pyodbc

In [38]:
#Lee el archivo de los proveedores
def leer_archivos():
    
    #Especificar la ulr para agarrar todos los archivos de ahi, de momento no tengo la url de una compartida entonces por el momento no importa
    #url = ""
    nombre_cts = "Copia de CTS_Procesamiento_2025.06.16.xlsx"
    nombre_int = "Copia de Procesamiento_compras_INTENEG_2025-06-16.xlsx"
    
    url_cts = nombre_cts
    url_int = nombre_int
    
    dtype = {
        'Cuenta' : str,
        'Tarjeta' : str,
        'Autorizacion1' : str
    }

    #Lee los archivos tal cual como vienen en el sharepoint
    df_cts = pd.read_excel(url_cts, dtype=dtype, parse_dates=["Fcompra1"])
    df_int = pd.read_excel(url_int, dtype = dtype, parse_dates=["Fcompra1"])
    
    df_cts["Fcompra1"] = df_cts["Fcompra1"].dt.strftime("%d/%m/%Y")
    df_int["Fcompra1"] = df_int["Fcompra1"].dt.strftime("%d/%m/%Y")
    
    
    #Esto junta los dos df en uno solo pero no se si es necesario tenerlos separados o si si se puede tenerlos juntos
    df_prove = pd.concat([df_cts,df_int])
    
    return df_prove

def quitar_duplicados(df):

    #Crea la llave
    df["Llave"] = df["Cuenta"] + df["Autorizacion1"]

    #Borramos los duplicados
    df = df.drop_duplicates(subset = 'Llave')
    
    return df
    
#def quitar_historicos(df):
    #Eliminamos las que ya se encuentrann en el historico
    #df_prove = df_prove[~df_prove['Llave'].isin(set(df_historico['Llave']))]

def formato_SQL(df,columna):
    
    #Convertir columnas en formato SQL (con comillas y coma) para la consulta SQL ejemplo 123456 va a ser igual a '123456',
    lista_formato_SQL = df[columna].tolist()
    lista_formato_SQL= ', '.join(f"'{tarjeta}'" for tarjeta in lista_formato_SQL)
    
    return lista_formato_SQL
    
    
def teradata_sebas(consulta_por):
    query = f"""SELECT  
        substr(NUM_CAR_CARF,4,16) as PAN
        ,NUM_CTA_CTAF as CTA 
        ,VAL_TRANS_COM as IMPORTE
        ,DAT_TRANS_COM as FECHA_TXN
        ,NOM_ESTB_FPRO as NOMBRE_COMERCIO
        ,NUM_REF_COM as REF_TXN
        ,NOM_FANT_CARF as NOMBRE_CLIENTE
        ,substr(X.VAL_DADO_ORIG_LKP,4,5) AS PLAN 
        ,NUM_AUT_COM
        ,DAT_PROX_CORTE_FAT_ATRF AS Prox_Corte
        ,COD_TIPO_TRANSACAO_FPRO

        FROM CARTAO_FINANCIAMENTO_V	

        INNER JOIN CONTA_FINANCIAMENTO_V
        ON ID_CTA_CARF = ID_CTA_CTAF

        LEFT OUTER JOIN COMPRA_V
        ON ID_CAR_COM = ID_CAR_CARF

        INNER JOIN PLANO_CREDITO_V X
        ON ID_PLANO_CRED_COM = X.ID_PLANO_CRED_LKP

        INNER JOIN TIPO_TRANSACAO_V
        ON ID_TIPO_TRANS_COM = ID_TIPO_TRANS_LKP

        LEFT OUTER JOIN FAT_TRANSACAO_PROSA_V
        ON ID_CAR_FPRO = ID_CAR_COM
        AND NUM_REF_FPRO = NUM_REF_COM

        inner join ATRIBUTO_CONTA_FINANCIAMENTO_V
        on ID_CTA_CTAF = ID_CTA_ATRF and current_date between DAT_DE_ATRF and DAT_ATE_ATRF

        WHERE
        {consulta_por}
        and DAT_TRANS_COM >= CURRENT_DATE - INTERVAL '60' DAY
        and substr(X.VAL_DADO_ORIG_LKP,4,5) in ('01000','01100','01181','06000')

    
    """
    
    return query


def conexion_a_teradata():
    
    try:
        print('Iniciando sesión en Teradata...')
    
        host: str = '192.168.17.64'  # Dirección IP o nombre del servidor Teradata
        user: str = 'MPROC_GR'       # Usuario de Teradata
        password: str = 'padrao' # Contraseña de Teradata
        driver: str = "Teradata Database ODBC Driver 20.00"  # Controlador ODBC

        udaExec = teradata.UdaExec(appName="PythonApp", version="1.0", logConsole=False)

        session = udaExec.connect(

            method="odbc",
            system=host,
            username=user,
            password=password,
            driver=driver,
            sslmode='Allow'
        )
        
    except Exception as e:
        print(f"Error en la conexión a Teradata: {str(e)}")
        return None  # Retorna None en caso de error
    
    return session

    
def hacer_consulta(session, query):
    
    session.execute("DATABASE PM_MstrDB")
    df = pd.read_sql(query, session)
    
    return df

def cerrar_conexion(session):
    session.close()

    
def cruces(df,df_teradata,parametro_teradata,parametro_df,corregir_cuentas):
    
    #Crear una llave en terada cuenta&aut&monto&fecha o tarjeta&aut&monto&fecha depende del parametro
    df_teradata["Llave_completa"] = df_teradata[parametro_teradata] + df_teradata["NUM_AUT_COM"] + df_teradata["IMPORTE"].astype(str) + df_teradata["FECHA_TXN"].astype(str) + df_teradata["PLAN"].astype(str)
    df["Llave_completa"] = df[parametro_df] + df["Autorizacion1"] + df["Monto1"].astype(str) + df["Fcompra1"].astype(str) + df["Txn1"].astype(str)
    
    #Busca en las dos consultas de teradata si se encuentran esos casos y asi saber que todos los campos de la llave estan bien
    df_bien = df[df['Llave_completa'].isin(set(df_teradata['Llave_completa']))]
    
    #Cuando las cuentas estan mal (se hizo la consulta por tarjeta y coincidieron casos), asignar la cuenta de teradata
    if corregir_cuentas == True:
        df_bien = pd.merge(df_bien, df_teradata[["Llave_completa", "CTA"]], on="Llave_completa", how="left")
        df_bien["Cuenta"] = df_bien["CTA"]
        df_bien.drop(['CTA'], axis=1, inplace=True)
        
    
    #Los que no tienen algun campo bien, averiguar que campo esta mal y poner el correcto?
    df_mal = df[~df['Llave_completa'].isin(set(df_teradata['Llave_completa']))]
    
    return df_bien, df_mal
    
    
def correccion_datos(df_mal, df_teradata, parametro_teradata, parametro_df):
    
    #Crea la llave corta en teradata cuenta&aut o tarjeta&aut
    df_teradata["Llave"] = df_teradata[parametro_teradata] + df_teradata["NUM_AUT_COM"]
    df_mal["Llave"] = df_mal[parametro_df] + df_mal["Autorizacion1"]
    
    df_no_estan = df_mal[~df_mal["Llave"].isin(set(df_teradata['Llave']))]
    df_mal = df_mal[df_mal["Llave"].isin(set(df_teradata['Llave']))]
    
    df_mal = pd.merge(df_mal, df_teradata[["Llave", "CTA", "IMPORTE", "FECHA_TXN", "PLAN"]], on="Llave", how="left")
    
    df_mal["Cuenta"] = df_mal["CTA"]
    df_mal["Monto1"] = df_mal["IMPORTE"]
    df_mal["Fcompra1"] = df_mal["FECHA_TXN"]
    df_mal["Txn1"] = df_mal["PLAN"]

    df_mal.drop(['IMPORTE', 'FECHA_TXN', 'PLAN', 'CTA'], axis=1, inplace=True)
    
    return df_mal,df_no_estan
    
def consultar_teradata(consulta_por):
            
    #Hace la consulta en teradata hace la consulta por cuentas, se indica si se consulta por cuenta o por tarjeta
    session = conexion_a_teradata()
    df_teradata = hacer_consulta(session, teradata_sebas(consulta_por))
    cerrar_conexion(session)
   

    #Cambiar el formato de la fecha a dd/mm/aaaa
    df_teradata['FECHA_TXN'] = pd.to_datetime(df_teradata['FECHA_TXN'], dayfirst=True, errors='coerce')
    df_teradata['FECHA_TXN'] = df_teradata['FECHA_TXN'].dt.strftime('%d/%m/%Y')
    df_teradata['IMPORTE'] = df_teradata['IMPORTE'].map(lambda x: ('%.2f' % x).rstrip('0').rstrip('.') if '.' in ('%.2f' % x) else '%.2f' % x)
    df_teradata["PLAN"] = df_teradata["PLAN"].astype(int)
    df_teradata["PAN"] = "000" + df_teradata["PAN"].astype(str)
    
    
    return df_teradata

def exportar_archivo(df, df_bien, df_mal, df_cuentas_mal, df_cuen_fech_impo_mal, df_no_estan, df_corregidos, url):
    with pd.ExcelWriter(url) as writer:
        df.to_excel(writer, "Completo", index=False)
        df_bien.to_excel(writer, "Casos Correctos", index=False)
        df_mal.to_excel(writer, "Casos Fecha, Importe, Plan mal", index=False)
        df_cuentas_mal.to_excel(writer, "Casos cuenta mal", index=False)
        df_cuen_fech_impo_mal.to_excel(writer, "Todo mal", index=False)
        df_no_estan.to_excel(writer,"No estan en Teradata", index=False)
        df_corregidos.to_excel(writer, "Todos corregidos", index=False)
        
    print("Archivo guardado con exito")

    
    
def proceso(df):
    #Esta funcion servira por si tenemos que mantener los df separados entonces pasamos los df por esta funcion
    #y ya no escribir cada paso dos veces
    
    df = quitar_duplicados(df)
    
    #Convierte todas las cuentas en formato SQL
    cuentas = formato_SQL(df, "Cuenta")
    tarjetas = formato_SQL(df,"Tarjeta")
    
    #Linea que sirve para complementar el query para saber por que consultar
    consultar_por_cuenta = f"NUM_CTA_CTAF IN ({cuentas})"
    consultar_por_tarjeta = f"NUM_CAR_CARF IN ({tarjetas})"
    
    #Quita los 0 que no nos sirven
    df['Monto1'] = df['Monto1'].map(lambda x: ('%.2f' % x).rstrip('0').rstrip('.') if '.' in ('%.2f' % x) else '%.2f' % x)
    
    #Vamos a hacer la consulta de teradata por cuenta, por lo que el segundo parametro no importa ahorita (no importa la tarjeta por eso los 0000000)
    df_teradata = consultar_teradata(consultar_por_cuenta)
    
    #Hace los cruces con teradata usando la llave completa, cuenta como uno de los parametros
    #En caso de que no encuentre en teradata, hace los cruces con cuenta&aut y asigna las fehcas y montos
    #Identifica los que aun asi  noestan
    #Se indica que no se van a corregir las cuentas con el false
    df_bien, df_mal = cruces(df, df_teradata, "CTA", "Cuenta", False)
    df_mal,df_no_estan = correccion_datos(df_mal,df_teradata, "CTA", "Cuenta")
    
    #Hace los cruces con teradata usando la llave completa, tarjeta como uno de los parametros
    #En caso de que no encuentre en teradata, hace los cruces con tarjeta&aut y asigna las fehcas y montos
    #Identifica los que aun asi  noestan
    #Vamos a hacer la consulta de teradata por tarjeta, por lo que el segundo parametro no importa ahorita (no importa la cuenta por eso los 0000000)
    #Se indica que no se van a corregir las cuentas con el false
    df_teradata = consultar_teradata(consultar_por_tarjeta)    
    df_cuentas_mal, df_cuen_fech_impo_mal = cruces(df_no_estan, df_teradata, "PAN", "Tarjeta", True)
    df_cuen_fech_impo_mal,df_no_estan = correccion_datos(df_cuen_fech_impo_mal,df_teradata, "PAN", "Tarjeta")
    
    #Concatenar todos los df corregidos en uno solo (menos el completo y los manuales) 
    df_corregidos = pd.concat([
        df_bien,
        df_mal,
        df_cuentas_mal,
        df_cuen_fech_impo_mal,
    ], ignore_index=True)
    
    url = "C:\\Users\\luis.campos\\Downloads\\Parcializaciones\\casos.xlsx"
    
    #Exporta el archivo con todos los df, se hacen los casos manuales y se agregan a la hoja de "Todos corregidos" del excel, luego compara todos con el historico
    exportar_archivo(df, df_bien, df_mal, df_cuentas_mal, df_cuen_fech_impo_mal, df_no_estan, df_corregidos, url)
    
    input("CORRIJA Y AGREGUE LOS CASOS MANUALES / APRIETE CUALQUIER TECLA PARA CONTINUAR")
    print("Cargando")
    
    df_corregidos = comparar_con_historico(df_corregidos)
    
    exportar_archivo(df, df_bien, df_mal, df_cuentas_mal, df_cuen_fech_impo_mal, df_no_estan, df_corregidos, url)

    
    

    
def comparar_con_historico(df):
    
    nombre_historico = "historico (1).xlsx"
    #Esto es por si no se encuentra en la ruta del codigo
    url_historico = nombre_historico
    
    dtype = {
        "Cuenta" : str,
        "Tarjeta" : str,
        "Autorizacion1": str,
        "PS" : str,
        "Store" : str,
    }
    
    
    #Que el historico sea de una sola hoja, no separada por socios
    df_historico = pd.read_excel(url_historico, dtype=dtype, parse_dates=["Fcompra1"])

    #Poner las columnas fecha e importe con el mismo formato a los del df
    df_historico['Fcompra1'] = pd.to_datetime(df_historico['Fcompra1'], dayfirst=True, errors='coerce')
    df_historico['Fcompra1'] = df_historico['Fcompra1'].dt.strftime('%d/%m/%Y')
    df_historico['Monto1'] = df_historico['Monto1'].map(lambda x: ('%.2f' % x).rstrip('0').rstrip('.') if '.' in ('%.2f' % x) else '%.2f' % x)
    
    
    #Crear llave completa para el historico
    df_historico["Llave_completa"] = df_historico["Cuenta"] + df_historico["Autorizacion1"] + df_historico["Monto1"].astype(str) + df_historico["Fcompra1"].astype(str) + df_historico["Txn1"].astype(str)
    
    #Elimina del df los que estan en el historico
    df = df[~df["Llave_completa"].isin(set(df_historico['Llave_completa']))]
    
    exportar_historico(df,df_historico)
    
    return df
    
def exportar_historico(df, df_historico):
    
    nombre_historico = "Historico.xlsx"
    #Esto es por si no se encuentra en la ruta del codigo
    url_historico = nombre_historico
    
    df_hoy = df.drop(['Llave',  'Llave_completa'], axis=1)
    df_historico = df_historico.drop("Llave_completa", axis=1)
    df_historico = pd.concat([df_hoy, df_historico], ignore_index=True)
    
    with pd.ExcelWriter(url_historico) as writer:
        df_historico.to_excel(writer, "Historico", index=False)

    print("Historico guardado con exito")
    
    
def main():
    
    
    #Para no hacer todo dos veces en caso de qu los df se deban mantener separados, se puede crear una funcion en donde se ponga 
    #todos los pasos y de parametro reciba un df, finalmente en el main llamamos a esa funcio dos veces una por cada df
    df_prove = leer_archivos()
    proceso(df_prove)
    
    #print(df_bien)
    
    
    
    
main()



C:\Users\luis.campos\AppData\Local\Temp\ipykernel_12436\1746781821.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Monto1'] = df['Monto1'].map(lambda x: ('%.2f' % x).rstrip('0').rstrip('.') if '.' in ('%.2f' % x) else '%.2f' % x)


Iniciando sesión en Teradata...


C:\Users\luis.campos\AppData\Local\Temp\ipykernel_12436\1746781821.py:132: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session)
C:\Users\luis.campos\AppData\Local\Temp\ipykernel_12436\1746781821.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Llave_completa"] = df[parametro_df] + df["Autorizacion1"] + df["Monto1"].astype(str) + df["Fcompra1"].astype(str) + df["Txn1"].astype(str)
C:\Users\luis.campos\AppData\Local\Temp\ipykernel_12436\1746781821.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

Iniciando sesión en Teradata...


C:\Users\luis.campos\AppData\Local\Temp\ipykernel_12436\1746781821.py:132: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session)
C:\Users\luis.campos\AppData\Local\Temp\ipykernel_12436\1746781821.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mal["Llave"] = df_mal[parametro_df] + df_mal["Autorizacion1"]


Archivo guardado con exito
CORRIJA Y AGREGUE LOS CASOS MANUALES / APRIETE CUALQUIER TECLA PARA CONTINUAR
Cargando
                    Cuenta              Tarjeta  \
4141   0004037500077005000  0004037500077005013   
4142   0004037500077005000  0004037500077005013   
4143   0004037500077005000  0004037500077005013   
4144   0004037500089660000  0004037500089660011   
4145   0004037500089660000  0004037500089660011   
...                    ...                  ...   
9997   0004812830930448000  0004812831005308943   
9998   0004812830930448000  0004812831005308943   
9999   0004812830930448000  0004812831005308943   
10000  0004812830930448000  0004812831005308943   
10001  0004812830893425000  0004812831008273847   

                               Nombre Autorizacion1   Monto1    Fcompra1  \
4141          FLORES MARTINEZ AZUCENA        052329      529  10/06/2025   
4142          FLORES MARTINEZ AZUCENA        001178    429.5  08/06/2025   
4143          FLORES MARTINEZ AZUCENA        

Historico guardado con exito
Archivo guardado con exito
